## Load Data
### Uber Eats Data

In [16]:
from sql_reader import connect_to_sql_server, read_dataframe_from_sql

In [17]:
conn, cursor = connect_to_sql_server()

In [18]:
query = """
SELECT m.id AS item_id, m.menu_name AS item_name, m.menu_category, m.menu_item_description, m.restaurant_name, m.restaurant_id, r.restaurant_type
FROM Menu_mds_sorted m
JOIN Restaurants_mds r ON m.restaurant_id = r.id
WHERE m.restaurant_id%1500 = 0
"""
df_uber = read_dataframe_from_sql(query, conn)
df_uber.head()

/Users/zhiweizhang/Documents/Canada/UBC/capstone/heymate-report/script/sql_reader.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Read 2281 rows from SQL Server.


,item_id,item_name,menu_category,menu_item_description,restaurant_name,restaurant_id,restaurant_type
0,4500-1,Crunchwrap Supreme® Combo,Combos,Served with a large drink and a regular crunch...,Taco Bell (315 West Avenue North),4500,"Burritos, Fast Food, Mexican"
1,4500-2,Crunchwrap Supreme®,Picked for you,"A flour tortilla layered with seasoned beef, w...",Taco Bell (315 West Avenue North),4500,"Burritos, Fast Food, Mexican"
2,4500-3,Quesarito,Picked for you,"A melty cheese quesadilla, with shredded chedd...",Taco Bell (315 West Avenue North),4500,"Burritos, Fast Food, Mexican"
3,4500-4,Cinnabon Delights® 12 Pack,Picked for you,Warm donut holes filled with Cinnabon® signatu...,Taco Bell (315 West Avenue North),4500,"Burritos, Fast Food, Mexican"
4,4500-5,Chicken Quesadilla,Picked for you,An extra flour tortilla filled with melty thre...,Taco Bell (315 West Avenue North),4500,"Burritos, Fast Food, Mexican"


### Heymate Internal Data

In [6]:
query = """
SELECT *
FROM Internal_menu_mds
"""
df_heymate = read_dataframe_from_sql(query, conn)
conn.close()
df_heymate.head()

Read 4866 rows from SQL Server.


/Users/zhiweizhang/Documents/Canada/UBC/capstone/heymate-report/script/sql_reader.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,item_id,row_id,item_name,menu_category,restaurant_name,restaurant_id,restaurant_type
0,459-1,1,Spicy Combo Spicy Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
1,459-2,21,Dynamite Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
2,459-3,32,Kids Roll Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
3,459-4,43,Salmon/Tuna Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
4,459-5,54,Chef's Choice Sushi (10pcs),Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant


## LLM Data Cleaning

In [19]:
from openai import OpenAI
from datetime import datetime, timezone
import requests
import time
import json
from typing import Iterable, List, Dict, Any, Union
import pandas as pd

### Configuration

In [20]:
ALLOWED_RESTAURANT_TYPES = [
    "acai shop", "afghani restaurant", "african restaurant", "american restaurant", "asian restaurant",
    "bagel shop", "bakery", "bar", "bar and grill", "barbecue restaurant", "brazilian restaurant",
    "breakfast restaurant", "brunch restaurant", "buffet restaurant", "cafe", "cafeteria", "candy store",
    "cat cafe", "chinese restaurant", "chocolate factory", "chocolate shop", "coffee shop", "confectionery",
    "deli", "dessert restaurant", "dessert shop", "diner", "dog cafe", "donut shop", "fast food restaurant",
    "fine dining restaurant", "food court", "french restaurant", "greek restaurant", "hamburger restaurant",
    "ice cream shop", "indian restaurant", "indonesian restaurant", "italian restaurant", "japanese restaurant",
    "juice shop", "korean restaurant", "lebanese restaurant", "meal delivery", "meal takeaway", "mediterranean restaurant",
    "mexican restaurant", "middle eastern restaurant", "pizza restaurant", "pub", "ramen restaurant", "restaurant",
    "sandwich shop", "seafood restaurant", "spanish restaurant", "steak house", "sushi restaurant", "tea house",
    "thai restaurant", "turkish restaurant", "vegan restaurant", "vegetarian restaurant", "vietnamese restaurant", "wine bar"
]

restaurant_type_list = ", ".join(f'"{r}"' for r in ALLOWED_RESTAURANT_TYPES)

In [21]:
MODEL_NAME = "o4-mini"
SYSTEM_PROMPT = """
You are a Menu Data Extractor.

Input: A list of menu items. Each item includes:
  - restaurant_name
  - restaurant_type
  - item_name
  - menu_item_description (may be empty)
  - menu_category (may be empty)

Output: a JSON array (same order) with:
  - dish_base : string (the primary dish name, e.g., "pizza")
  - dish_flavor : string[] (up to 5 flavour descriptors, e.g., "pepperoni")
  - is_combo : boolean
  - restaurant_type_std : string

Rules:
  • Use lowercase American English, and translate non-English terms into English based on Merriam-Webster and AP Stylebook.
  
  • dish_base:
      - Should be the main identity of the dish (e.g., "pizza", "lo mein", "fried rice").
      - Remove size indicators (e.g., "small", "XL"), quantity counts (e.g., "3 pcs"), and side items.
      - If the base is unclear or ambiguous, use "unknown".
      - Singular form only (e.g., "sandwich" not "sandwiches").
      - Use menu_name and context from other inputs to identify dish_base.

  • dish_flavor:
      - Up to 5 descriptors of flavor, cooking style, toppings, sauces, etc.
      - Each tag must be no more than two words.
      - DO NOT repeat dish_base (e.g., if dish_base is "shrimp fried rice", do not include "fried rice" again) unless meaning is added.
      - Use singular form (e.g., "egg" not "eggs")and lowercase.

  • is_combo:
      - Set to `true` if the item clearly bundles multiple components (e.g., main dish + sides + drink).

  • restaurant_type_std:
      - Must match one of the following values exactly: {restaurant_type_list}
      - Use the exact spelling and spacing and do NOT add or modify words (e.g., don't add "restaurant" if it's not part of the allowed type).
      - Start by checking whether the input field `restaurant_type` contains or closely matches any of the allowed types.
        - If it partially matches or contains keywords, normalize it accordingly (e.g., "fast food, burgers" → "fast food restaurant").
      - If no reliable match is found from `restaurant_type`, use other inputs to infer. 

      
  • Only return raw JSON (no Markdown, no backticks (```), no extra labels). Output must be valid JSON and preserve the original input order.

Example input:
{
    "restaurant_name": "Pizza World",
    "restaurant_type": "Fast Food, Pizza, Burgers",
    "item_name": "Bacon Cheeseburger Combo",
    "menu_item_description": "Cheeseburger with bacon, fries, and a large coke.",
    "menu_category": "Combo"
  },
  {
    "restaurant_name": "Sushi & Wok",
    "restaurant_type": "Japanese, Chinese",
    "item_name": "Shrimp Fried Rice",
    "menu_item_description": "Grilled shrimp with soy glaze, served over fried rice.",
    "menu_category": ""
  }

Expected output:
[
  {
    "dish_base": "cheeseburger",
    "dish_flavor": ["bacon", "fries", "coke"],
    "is_combo": true,
    "restaurant_type_std": "fast food restaurant"
  },
  {
    "dish_base": "fried rice",
    "dish_flavor": ["shrimp", "grilled", "soy glaze"],
    "is_combo": false,
    "restaurant_type_std": "chinese restaurant"
  }
]
"""

In [34]:
class OpenAIConnector:
    def __init__(self, token_path="../credentials/open_ai_token.txt"):
        self.token = self._load_token(token_path)
        self.client = OpenAI(api_key=self.token)
        self.current_prompt = ''

    @staticmethod
    def _load_token(token_path):
        try:
            with open(token_path, "r") as f:
                token = f.read().strip()
                if not token:
                    raise ValueError("Token file is empty.")
                return token
        except FileNotFoundError:
            raise FileNotFoundError(f"Token file not found at {token_path}")
        except Exception as e:
            raise RuntimeError(f"Error reading token: {e}")
            
    # ─────────────────────────────────────────────────────────────
    #  PUBLIC –  main entry point
    # ─────────────────────────────────────────────────────────────
   
    def classify_batch(
        self,
        rows: Union[str, Iterable[Dict[str, str]]],
        model: str = MODEL_NAME,
        timeout_s: int = 60
    ) -> List[Dict[str, Any]]:
        start_time = time.time()
    
        if isinstance(rows, str):
            rows = json.loads(rows)
    
        # Make the message readable, clean JSON string
        user_msg = json.dumps(rows, indent=2)
    
        # Send one request for the entire batch
        resp = self.client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_msg},
            ],
            timeout=timeout_s
        )
    
        # Parse full JSON array
        content = resp.choices[0].message.content
        try:
            results = json.loads(content)
        except json.JSONDecodeError:
            raise ValueError("❌ Failed to parse response as JSON:\n" + content)
    
        # elapsed = time.time() - start_time
        # print(f"✅ Processed {len(rows)} rows in {elapsed:.2f} seconds.")
        # print(f"⏱️ Average time per row: {elapsed / len(rows):.2f} seconds.")
    
        return results

        
    def get_usage_summary(self):
        headers = {
            "Authorization": f"Bearer {self.token}"
        }

        now = datetime.now(timezone.utc)
        start_date = now.replace(day=1).strftime("%Y-%m-%d")
        end_date = now.strftime("%Y-%m-%d")

        # 1. Get usage data
        usage_url = f"https://api.openai.com/v1/dashboard/billing/usage?start_date={start_date}&end_date={end_date}"
        usage_resp = requests.get(usage_url, headers=headers)
        usage_data = usage_resp.json()
        used_usd = usage_data.get("total_usage", 0) / 100.0  # from cents to dollars

        # 2. Get allowance (subscription limit)
        limits_url = "https://api.openai.com/v1/dashboard/billing/subscription"
        limits_resp = requests.get(limits_url, headers=headers)
        limits_data = limits_resp.json()
        hard_limit = limits_data.get("hard_limit_usd", 0)
        soft_limit = limits_data.get("soft_limit_usd", 0)

        remaining = hard_limit - used_usd

        return {
            "used_usd": round(used_usd, 2),
            "soft_limit_usd": round(soft_limit, 2),
            "hard_limit_usd": round(hard_limit, 2),
            "remaining_usd": round(remaining, 2)
        }

### Test on Sample Batch

In [35]:
import random

def qc(df_input, batch_index=None, batch_size=30,  col_mapping=None, allowed_types=None):
    if col_mapping is None:
        col_mapping = {}

    df = df_input.rename(columns=col_mapping).copy()

    # Fill missing expected cols with empty string
    expected_cols = ["item_id", "restaurant_name", "restaurant_type", "item_name", "menu_item_description", "menu_category"]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = ""

    if batch_index is None:
        batch_index = random.randint(0, len(df_input) // batch_size) 

    df_batch = df[expected_cols]
    df_batch = df_batch[batch_size * batch_index : batch_size * (batch_index + 1)]

    # Initialize connector and classify
    oai = OpenAIConnector()
    result = oai.classify_batch(df_batch.to_dict(orient='records'))

    # Combine with original
    output_json = []
    for item_id, res in zip(df_batch["item_id"].values, result):
        res = res or {}
        record = {
            "item_id": item_id,
            "dish_base": res.get("dish_base", ""),
            "dish_flavor": res.get("dish_flavor", []),
            "is_combo": res.get("is_combo", False),
            "restaurant_type_std": res.get("restaurant_type_std", "")
        }
        output_json.append(record)

    # Test on restaurant type
    if allowed_types is not None:
        found_types = set(str(r.get("restaurant_type_std", "")).strip().lower() for r in output_json)
        allowed_set = set(v.strip().lower() for v in allowed_types)
        invalid_values = found_types - allowed_set
        if invalid_values:
            raise ValueError(f"Invalid restaurant_type_std values found: {sorted(invalid_values)}")

    return output_json

In [36]:
# Test of Uber Eats data
result_uber = qc(df_uber, batch_index=23, batch_size=10, allowed_types=ALLOWED_RESTAURANT_TYPES)
result_uber

[{'item_id': '6000-80',
  'dish_base': 'salt',
  'dish_flavor': [],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-81',
  'dish_base': 'splenda',
  'dish_flavor': [],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-82',
  'dish_base': 'dipping sauce',
  'dish_flavor': ['tangy', 'bbq'],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-83',
  'dish_base': 'ketchup',
  'dish_flavor': [],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-84',
  'dish_base': 'dipping sauce',
  'dish_flavor': ['sweet-and-sour'],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-85',
  'dish_base': 'honey',
  'dish_flavor': [],
  'is_combo': False,
  'restaurant_type_std': 'fast food restaurant'},
 {'item_id': '6000-86',
  'dish_base': 'ranch sauce',
  'dish_flavor': ['creamy'],
  'is_combo': False,
  'rest

In [37]:
# Test of heymate data
result_heymate = qc(df_heymate, batch_index=40, batch_size=20, allowed_types=ALLOWED_RESTAURANT_TYPES)
result_heymate

[{'item_id': '526-68',
  'dish_base': 'rice bowl',
  'dish_flavor': ['chicken', 'teriyaki'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-69',
  'dish_base': 'rice bowl',
  'dish_flavor': ['beef', 'teriyaki'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-70',
  'dish_base': 'rice bowl',
  'dish_flavor': ['oyako'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-71',
  'dish_base': 'rice bowl',
  'dish_flavor': ['katsu'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-72',
  'dish_base': 'rice bowl',
  'dish_flavor': ['tempura'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-73',
  'dish_base': 'rice bowl',
  'dish_flavor': ['chicken karaage', 'soy'],
  'is_combo': False,
  'restaurant_type_std': 'sushi restaurant'},
 {'item_id': '526-74',
  'dish_base': 'rice bowl',
  'dish_flavor': ['chicken ka